# 1. Ler o DataSet

In [1]:
import pandas as pd
import os

filename = 'dataset.xlsx'

print(f'Esperamos trabalhar no diretório: \n{os.getcwd()}\n')

if filename in os.listdir():
    print(f'Parece que o arquivo {filename} está na mesma pasta do notebook, yay!')
    
else:
    print(f'Não encontrei o arquivo {filename}')

Esperamos trabalhar no diretório: 
/Users/fuchsfelipel/code/insper/projeto-2

Parece que o arquivo dataset.xlsx está na mesma pasta do notebook, yay!


In [2]:
dados = pd.read_excel(filename)
dados

,price (£),number_of_reviews,rating,category
0,3.42,15,4.9,Hobbies > Model Trains & Railway Sets > Rail V...
1,16.99,2,4.5,Hobbies > Model Trains & Railway Sets > Rail V...
2,9.99,17,3.9,Hobbies > Model Trains & Railway Sets > Rail V...
3,39.99,1,5.0,Hobbies > Model Trains & Railway Sets > Rail V...
4,32.19,3,4.7,Hobbies > Model Trains & Railway Sets > Rail V...
...,...,...,...,...
7998,22.95,3,5.0,Hobbies > Collectible Figures & Memorabilia > ...
7999,39.99,1,4.0,Characters & Brands > Star Wars > Toys
8000,43.99,1,5.0,Novelty & Special Use > Novelty > Accessories ...
8001,49.81,1,5.0,Hobbies > Collectible Figures & Memorabilia > ...


# 2. Limpar o DataSet

In [3]:
print(f"Temos as colunas {dados.columns}")

dados = dados.drop('category', 1)
dados = dados.drop('number_of_reviews', 1)

print(f"Agora temos as colunas {dados.columns}")

Temos as colunas Index(['price (£)', 'number_of_reviews', 'rating', 'category'], dtype='object')
Agora temos as colunas Index(['price (£)', 'rating'], dtype='object')


/var/folders/gj/1r66hhws0pxggh3y_j668sfr0000gn/T/ipykernel_12275/3700944776.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dados = dados.drop('category', 1)
/var/folders/gj/1r66hhws0pxggh3y_j668sfr0000gn/T/ipykernel_12275/3700944776.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dados = dados.drop('number_of_reviews', 1)
